In [6]:
best_run_info_run_id = "sweet_bell_qc73gvv9m6_HD_3"

In [2]:
import os
import urllib
from IPython.display import Image
from zipfile import ZipFile
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from azure.ai.ml import MLClient
from azure.ai.ml import automl
from azure.identity import DefaultAzureCredential

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from PIL import Image
import jsonlines

import json
import mlflow
from mlflow.tracking.client import MlflowClient
from azure.ai.ml.entities import (ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment,CodeConfiguration,ProbeSettings)

def get_azure_credential(subscription_id, resource_group, workspace):
    credential = DefaultAzureCredential()
    ml_client = MLClient.from_config(credential)
    ml_client =  MLClient(credential, subscription_id, resource_group, workspace)
    return ml_client

ml_client = get_azure_credential("","","")

Found the config file in: /config.json


In [3]:
import datetime
online_endpoint_name = "lemon-items" + datetime.datetime.now().strftime("%m%d%H%M")

endpoint = ManagedOnlineEndpoint(
    name = online_endpoint_name,
    description="this is a sample of a endpoint",
    auth_mode="key",
    tags={"lemons":"quality"}
)

In [4]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://lemon-items11291500.westus.inference.ml.azure.com/score', 'openapi_uri': 'https://lemon-items11291500.westus.inference.ml.azure.com/swagger.json', 'name': 'lemon-items11291500', 'description': 'this is a sample of a endpoint', 'tags': {'lemons': 'quality'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourcegroups/rchoque/providers/microsoft.machinelearningservices/workspaces/ws_ml_rcf/onlineendpoints/lemon-items11291500', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/oe:7997d505-3695-4109-8af5-d01005f1102f:ae52929a-e69a-4b5b-af5c-8f25b0b59311?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourceGroups/rchoqu

In [7]:
model_name = "lemon-detection-model"
model = Model(
    path=f"azureml://jobs/{best_run_info_run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description = "my sample image",
    type= AssetTypes.MLFLOW_MODEL,
)
model

Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': True, 'auto_delete_setting': None, 'name': 'lemon-detection-model', 'description': 'my sample image', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/testnotebookrcf/code/Users/ron.todo.poderoso/Proyecto', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fdc888cebe0>, 'version': None, 'latest_version': None, 'path': 'azureml://jobs/sweet_bell_qc73gvv9m6_HD_3/outputs/artifacts/outputs/mlflow-model/', 'datastore': None, 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'mlflow_model', 'stage': None})

In [8]:
registered_model = ml_client.models.create_or_update(model)

In [9]:
registered_model.id

'/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourceGroups/rchoque/providers/Microsoft.MachineLearningServices/workspaces/ws_ml_rcf/models/lemon-detection-model/versions/1'

In [10]:
from azure.ai.ml.entities import OnlineRequestSettings
req_timeout = OnlineRequestSettings(request_timeout_ms=90000)

deployment = ManagedOnlineDeployment(
    name = "id-mc-lemon-items-mlflow-depl",
    endpoint_name  = online_endpoint_name,
    model = registered_model.id,
    #gpu
    instance_type = "Standard_E4s_v3",
    instance_count = 1,
    request_settings = req_timeout
)

In [11]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint lemon-items11291500 exists


........................................................................................................